In [1]:
import sys
import os
import itertools
from tempfile import NamedTemporaryFile
from gc import collect
from time import time

import netCDF4
import numpy as np
import matplotlib.pyplot as plt
from osgeo import ogr

sys.path.append('../util')
from ncgen import make_nc
from meters import ThroughputMeter
from circle import wkt_circle
from geo import cartesian
from areas import metro_van_10, prov_bc_18, fp_270
from grids import bc_400m, canada_5k, world_125k, world_250k

from shapely import wkt
from shapely.geometry import Point


os.environ['TMPDIR'] = os.getenv('HOME') + '/tmp/'

In [2]:
def clipLatLonToPolyExtent(lat, lon, poly):
    minx, miny, maxx, maxy = poly.bounds    
    lat = lat[np.where( (lat > miny) & (lat < maxy) )]
    lon = lon[np.where( (lon > minx) & (lon < maxx) )]
    
    return lat, lon

def pointsInPoly(coords, poly):    
    x = [1 if Point(p[0], p[1]).within(poly) else 0 for p in coords]
    coords = coords[np.where(x)]
    return coords

In [3]:
def polygonToMask(nc, poly):
    nclats = nc.variables['lat'][:]
    nclons = nc.variables['lon'][:]

    mask = np.zeros(nc.variables['var_0'][0,:,:].shape)
    
    lat, lon = clipLatLonToPolyExtent(nclats, nclons, poly)
#     print(lat, lon)
    
    # If no centroids in polygon, return zeros
    if lat.size == 0 or lon.size == 0:
        return mask

#     print(lat, lon)
    coords = cartesian([lon, lat])
    pts = pointsInPoly(coords, poly)

    # map the lon, lat values back to numpy indices
    lons = np.sort(np.unique(pts[:,0]))
    lats = np.sort(np.unique(pts[:,1]))
    lon_indices = nclons.searchsorted(lons)
    lat_indices = nclats.searchsorted(lats)

    lat_map = dict(zip(lats, lat_indices))
    lon_map = dict(zip(lons, lon_indices))
    

    for pt in pts:
        mask[lon_map[pt[0]],lat_map[pt[1]]] = 1
        
    return mask

In [4]:
poly_complexity = [8, 16, 32, 64, 128]
grids = [world_250k, world_125k, canada_5k, bc_400m]


In [5]:
results = []

for grid in grids:
    with NamedTemporaryFile(suffix='.nc', delete=False, dir='/app/tmp') as f:
        nc = make_nc(f.name, grid=grid)
        
        # for sides in poly_complexity: #Circle generator doesn't seem to be working... circle centers near equator
        for area in metro_van_10, prov_bc_18, fp_270:
#           poly = wkt.loads(wkt_circle(sides))
            poly = wkt.loads(area)

            t0 = time()
            mask = polygonToMask(nc, poly)
            t1 = time()
            t = t1 - t0

            r = (len(poly.exterior.coords), np.sum(mask), t)
            print(r)
            results.append(r)

(10, 0.0, 0.0009949207305908203)
(19, 0.0, 0.0008440017700195312)
(271, 0.0, 0.0008258819580078125)
(10, 0.0, 0.0008127689361572266)
(19, 0.0, 0.0008101463317871094)
(271, 0.0, 0.0008289813995361328)
(10, 41.0, 0.005239963531494141)
(19, 20448.0, 1.7783362865447998)
(271, 1562.0, 0.5465691089630127)
(10, 0.0, 0.43389320373535156)
(19, 821098.0, 70.37642192840576)
(271, 10807.0, 10.277490854263306)


In [6]:
results

[(10, 0.0, 0.0009949207305908203),
 (19, 0.0, 0.0008440017700195312),
 (271, 0.0, 0.0008258819580078125),
 (10, 0.0, 0.0008127689361572266),
 (19, 0.0, 0.0008101463317871094),
 (271, 0.0, 0.0008289813995361328),
 (10, 41.0, 0.005239963531494141),
 (19, 20448.0, 1.7783362865447998),
 (271, 1562.0, 0.5465691089630127),
 (10, 0.0, 0.43389320373535156),
 (19, 821098.0, 70.37642192840576),
 (271, 10807.0, 10.277490854263306)]